In [1]:
import dspy
import pandas as pd
from pyprojroot import here
import uuid

In [2]:
turbo = dspy.OpenAI(model='gpt-4')
dspy.settings.configure(lm=turbo)

In [3]:
class VerbalAbstraction(dspy.Signature):
    """Classify the following sentences as one of either:
1. does not use abstract or analogical verbs
2. uses abstract or analogical verbs

These sentences all came from an experiment in which participants played a game with squares of different colors moving on a board then wrote about it.

Sentences that do not use abstract or analogical verbs could either talk about something other than the game or describe the game in terms of simple verbs. The following are examples of categories of verbs that should not be labelled as abstract or analogical:
- verbs that concretely describe the game environment, such as move, touch, push, disappear, appear, get rid of, change, and turn into.
- verbs that talk about basic game mechanics, like win, die, lose, or or achieve.
- verbs that describe the writer or reader's mental state, like understand or think.
- common verbs that show up in a lot of situations, like make and get.

Sentences use abstract or analogical verbs talk about more complex actions or make analogies to domains outside of the block world. The following are examples of abstract or analogical verb use:
- verbs referring to making blocks disappear in a certain way, like eat or destroy
- verbs that describe motion via analogy, like fall, drop, land, teleport, and follow.
- verbs that describe a resource bar, like charge and fill 

These are just some examples. There are many other ways verbs can be used in abstract or analogical ways.

Remember that this tagging is only about the verbs used in a sentence. The existence or lack of abstract nouns is not relevant.
"""

    sentence = dspy.InputField(desc="A sentence to be tagged")
    tag = dspy.OutputField(desc="'abstract' if the message contains abstract or analogical verbs and 'concrete' otherwise")

In [4]:
tag_renamer = {
    "concrete": "does not use abstract or analogical verbs",
    "abstract": "uses abstract or analogical verbs"
}

In [5]:
class TagAbstractVerbs(dspy.Module):
    """A module to tag verbs for abstraction"""
    def __init__(self):
        super().__init__()
        
        self.tagger = dspy.Predict(VerbalAbstraction)

    def forward(self, sentence):
        return self.tagger(sentence=sentence)

class TagAbstractVerbsWithCoT(dspy.Module):
    """A module to tag verbs for abstraction using chain of thought."""
    def __init__(self):
        super().__init__()
        
        self.tagger = dspy.ChainOfThought(VerbalAbstraction)

    def forward(self, sentence):
        return self.tagger(sentence=sentence)

In [7]:
df_tags.columns

Index(['sentence', 'tags'], dtype='object')

In [8]:
df_abstract_verbs["verb_tag"] = df_abstract_verbs["tags"].apply(lambda x: tag_renamer["abstract"] if "abstract_verbs:true" in x else tag_renamer["concrete"])

/var/folders/j2/m8twlxj12xdg_b0wm25qx_x40000gn/T/ipykernel_96165/3632953434.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_abstract_verbs["verb_tag"] = df_abstract_verbs["tags"].apply(lambda x: tag_renamer["abstract"] if "abstract_verbs:true" in x else tag_renamer["concrete"])


In [9]:
df_train, df_dev = df_abstract_verbs[:50], df_abstract_verbs[50:]
train_set = [dspy.Example({"sentence": row["sentence"], "tag": row["verb_tag"], "dspy_uuid": uuid.uuid4()}).with_inputs("sentence") for _, row in df_train.iterrows()]
dev_set = [dspy.Example({"sentence": row["sentence"], "tag": row["verb_tag"], "dspy_uuid": uuid.uuid4()}).with_inputs("sentence") for _, row in df_dev.iterrows()]

In [10]:
?dspy.evaluate.answer_exact_match

Object `dspy.evaluate.answer_exact_match` not found.


In [11]:
import dsp
from dspy.teleprompt import LabeledFewShot, BootstrapFewShot
from numpy.random import shuffle

def validate_tag(example, pred, trace=None, frac=1.0):
    print(f"example tag: {example.tag}")
    print(f"pred tag: {pred.tag}")
    print(f"tag result: {pred.tag == example.tag}")
    return pred.tag == example.tag
    
teleprompter = LabeledFewShot(k=20)

In [25]:
shuffle(train_set)
compiled_prompt = teleprompter.compile(student=TagAbstractVerbs(), trainset=train_set)

In [26]:
res = compiled_prompt("example sentence")

In [27]:
print(compiled_prompt)

tagger = Predict(VerbalAbstraction
- sentence = InputField(prefix=Sentence:, desc=A sentence to be tagged)
- tag = OutputField(prefix=Tag:, desc='abstract' if the message contains abstract or analogical verbs and 'concrete' otherwise))


Evaluate the accuracy of the compiled prompt

In [28]:
df_dev

,sentence,tags,verb_tag,predicted_tag
9,The reds turn yellow but not sure after that (...,relevant:true;valence:neutral;uncertainty:true...,does not use abstract or analogical verbs,does not use abstract or analogical verbs
11,Blue blocks disappear when touching a yellow b...,relevant:true;valence:neutral;uncertainty:fals...,does not use abstract or analogical verbs,does not use abstract or analogical verbs
12,The previous participant left me this message:...,relevant:true;valence:negative;uncertainty:fal...,does not use abstract or analogical verbs,uses abstract or analogical verbs
13,The light blue will move to cover all the gree...,relevant:true;valence:neutral;uncertainty:fals...,does not use abstract or analogical verbs,uses abstract or analogical verbs
14,you can fire fast so take a position at the si...,relevant:true;valence:neutral;uncertainty:fals...,uses abstract or analogical verbs,does not use abstract or analogical verbs
15,Then eat the second blue box.,relevant:true;valence:neutral;uncertainty:fals...,uses abstract or analogical verbs,uses abstract or analogical verbs
16,Think about how many light blue blocks you nee...,relevant:true;valence:neutral;uncertainty:fals...,does not use abstract or analogical verbs,uses abstract or analogical verbs
17,Avoid Yellow.,relevant:true;valence:neutral;uncertainty:fals...,does not use abstract or analogical verbs,does not use abstract or analogical verbs
18,Do no collect a green block if it is directly ...,relevant:true;valence:negative;uncertainty:fal...,uses abstract or analogical verbs,uses abstract or analogical verbs
19,There may be more than one route to getting to...,relevant:true;valence:positive;uncertainty:tru...,does not use abstract or analogical verbs,uses abstract or analogical verbs


In [29]:
df_dev["predicted_tag"] = df_dev["sentence"].apply(lambda x: compiled_prompt(x).tag)

/var/folders/j2/m8twlxj12xdg_b0wm25qx_x40000gn/T/ipykernel_86785/1696331415.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dev["predicted_tag"] = df_dev["sentence"].apply(lambda x: compiled_prompt(x).tag)


In [30]:
sum(df_dev["predicted_tag"] == df_dev["verb_tag"]) / len(df_dev)

0.5945945945945946

In [86]:
def inspect_example(df, i, column_name="predicted_tag"):
    row = df.iloc[i]
    print(f"sentence: {row['sentence']}")
    print(f"true tag: {row['verb_tag']}")
    print(f"predicted tag: {row[column_name]}")

In [87]:
inspect_example(df_dev, 7)

sentence: Avoid Yellow.
true tag: does not use abstract or analogical verbs
predicted tag: does not use abstract or analogical verbs


In [12]:
cot_teleprompter = BootstrapFewShot(metric=validate_tag)
compiled_cot_prompt = cot_teleprompter.compile(TagAbstractVerbsWithCoT(), trainset=train_set)

  2%|█▌                                                                              | 1/50 [00:04<03:37,  4.43s/it]

example tag: uses abstract or analogical verbs
pred tag: uses abstract or analogical verbs
tag result: True


  4%|███▏                                                                            | 2/50 [00:09<03:40,  4.59s/it]

example tag: uses abstract or analogical verbs
pred tag: does not use abstract or analogical verbs
tag result: False


  6%|████▊                                                                           | 3/50 [00:13<03:24,  4.34s/it]

example tag: does not use abstract or analogical verbs
pred tag: does not use abstract or analogical verbs
tag result: True


  8%|██████▍                                                                         | 4/50 [00:16<02:52,  3.75s/it]

example tag: does not use abstract or analogical verbs
pred tag: does not use abstract or analogical verbs
tag result: True


 10%|████████                                                                        | 5/50 [00:21<03:09,  4.21s/it]

example tag: does not use abstract or analogical verbs
pred tag: does not use abstract or analogical verbs
tag result: True
Bootstrapped 4 full traces after 6 examples in round 0.


In [13]:
turbo.inspect_history(n=1)





Classify the following sentences as one of either:
1. does not use abstract or analogical verbs
2. uses abstract or analogical verbs

These sentences all came from an experiment in which participants played a game with squares of different colors moving on a board then wrote about it.

Sentences that do not use abstract or analogical verbs could either talk about something other than the game or describe the game in terms of simple verbs. The following are examples of categories of verbs that should not be labelled as abstract or analogical:
- verbs that concretely describe the game environment, such as move, touch, push, disappear, appear, get rid of, change, and turn into.
- verbs that talk about basic game mechanics, like win, die, lose, or or achieve.
- verbs that describe the writer or reader's mental state, like understand or think.
- common verbs that show up in a lot of situations, like make and get.

Sentences use abstract or analogical verbs talk about more complex action

In [16]:
cot_results = df_dev["sentence"].apply(lambda x: compiled_cot_prompt(x))
df_dev["cot_predicted_tag"] = [r.tag for r in cot_results]
df_dev["cot_reasoning"] = [r.rationale for r in cot_results]

/var/folders/j2/m8twlxj12xdg_b0wm25qx_x40000gn/T/ipykernel_96165/1348093463.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dev["cot_predicted_tag"] = [r.tag for r in cot_results]
/var/folders/j2/m8twlxj12xdg_b0wm25qx_x40000gn/T/ipykernel_96165/1348093463.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dev["cot_reasoning"] = [r.rationale for r in cot_results]


In [17]:
sum(df_dev["cot_predicted_tag"] == df_dev["verb_tag"]) / len(df_dev)

0.6486486486486487

In [18]:
def inspect_reasoning(df, i, column_name="cot_predicted_tag"):
    row = df.iloc[i]
    print(f"sentence: {row['sentence']}")
    print(f"true tag: {row['verb_tag']}")
    print(f"predicted tag: {row[column_name]}")
    if row["verb_tag"] != row[column_name]:
        print(f"reasoning: {row['cot_reasoning']}")

In [19]:
for i in range(len(df_dev)):
    inspect_reasoning(df_dev, i, "cot_predicted_tag")
    print("\n")

sentence: The reds turn yellow but not sure after that (original message)
I don't get this one either I'm sorry.
true tag: does not use abstract or analogical verbs
predicted tag: does not use abstract or analogical verbs


sentence: Blue blocks disappear when touching a yellow block.
true tag: does not use abstract or analogical verbs
predicted tag: does not use abstract or analogical verbs


sentence: The previous participant left me this message: Do not touch/move the light blue blocks first or you will die.
true tag: does not use abstract or analogical verbs
predicted tag: does not use abstract or analogical verbs


sentence: The light blue will move to cover all the green squares and turn them purple.
true tag: does not use abstract or analogical verbs
predicted tag: does not use abstract or analogical verbs


sentence: you can fire fast so take a position at the side and fire one direction.
true tag: uses abstract or analogical verbs
predicted tag: does not use abstract or analog

In [ ]:
`